In [3]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
import IPython.display as display
from tqdm import tqdm
import random

In [55]:
!python read_data.py ../data/images/ ../data/face_data.tfrecord

이미지 갯수:  203


20220713221322.370 폴더 작업중: 100%|██████████| 94/94 [00:00<?, ?it/s]

20220713221444.370 폴더 작업중: 100%|██████████| 47/47 [00:00<?, ?it/s]

20220713221512.817 폴더 작업중: 100%|██████████| 23/23 [00:00<00:00, 23001.67it/s]

20220713221533.817 폴더 작업중: 100%|██████████| 39/39 [00:00<?, ?it/s]

  0%|          | 0/203 [00:00<?, ?it/s]2022-07-14 10:51:24.926211: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-14 10:51:25.598450: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2151 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5

100%|██████████| 203/203 [00:02<00:00, 81.62it/s] 


In [56]:
filenames = '../data/face_data.tfrecord'
raw_dataset = tf.data.TFRecordDataset(filenames)
raw_dataset
# Create a dictionary describing the features.
image_feature_description = {
    'image': tf.io.FixedLenFeature([], tf.string),    
    'class': tf.io.FixedLenFeature([], tf.int64),    
}

def _parse_image_function(example_proto):
  # Parse the input tf.train.Example proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, image_feature_description)

parsed_image_dataset = raw_dataset.map(_parse_image_function)
parsed_image_dataset

<MapDataset element_spec={'class': TensorSpec(shape=(), dtype=tf.int64, name=None), 'image': TensorSpec(shape=(), dtype=tf.string, name=None)}>

In [ ]:
for image_features in parsed_image_dataset:
    image_raw = image_features['image'].numpy()
display.display(display.Image(data=image_raw))